Pytorch + Bert tag classification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path
import os
import pandas as pd
import json
import numpy as np


Mounted at /content/drive


In [ ]:
BASE_DIR=Path().resolve().parent
GDRIVE_MOUNT=BASE_DIR/"gdrive"
DATA_DIR = GDRIVE_MOUNT /"MyDrive/Tag classification/"
INPUT_DIR = DATA_DIR / "input"

drive.mount(GDRIVE_MOUNT.as_posix(), force_remount=True)

Mounted at /gdrive


In [ ]:
NombaAccessISW=pd.read_csv(INPUT_DIR/ 'NombaAccessISW_FSP_2024.csv')
NombaAccessISW.head()

,id,transaction_date,value_date,transaction_type,amount,amount_currency,balance,balance_currency,matching_reference,reporting_tag,...,updated,fsp_accounts,fsp_transaction_ids,backend_accounts,backend_transaction_ids,other_accounts,other_transaction_ids,metadata,tags,description
0,490914812,2024-03-26 00:00:00.000000 UTC,2024-03-26 00:00:00.000000 UTC,C,5100.0,NGN,6.485760e+08,NGN,[3@2KUD3GWS2403261817305100.00],withdrawals,...,2024-06-27 07:54:12.000000 UTC,NaN,NaN,[430],[700497534],NaN,NaN,"{transaction_date=2024-03-26 00:00:00, origina...",[withdrawals],C|5100.0|2KUD3GWS-240326181730 LA LANG
1,490931395,2024-03-26 00:00:00.000000 UTC,2024-03-26 00:00:00.000000 UTC,C,50000.0,NGN,8.542449e+08,NGN,[3@2KUD121N24032619253050000.00],withdrawals,...,2024-06-27 07:54:12.000000 UTC,NaN,NaN,[430],[700731674],NaN,NaN,"{transaction_date=2024-03-26 00:00:00, origina...",[withdrawals],C|50000.0|2KUD121N-240326192530 LA LANG
2,490943811,2024-03-27 00:00:00.000000 UTC,2024-03-26 00:00:00.000000 UTC,C,5200.0,NGN,2.235371e+08,NGN,[3@2KUD164N2403262040145200.00],withdrawals,...,2024-06-27 07:54:12.000000 UTC,NaN,NaN,[430],[700356196],NaN,NaN,"{transaction_date=2024-03-27 00:00:00, origina...",[withdrawals],C|5200.0|2KUD164N-240326204014 LA LANG
3,490946685,2024-03-27 00:00:00.000000 UTC,2024-03-26 00:00:00.000000 UTC,C,2000.0,NGN,2.617250e+08,NGN,[3@2KUD65XQ2403262101012000.00],withdrawals,...,2024-06-27 07:54:12.000000 UTC,NaN,NaN,[430],[700335445],NaN,NaN,"{transaction_date=2024-03-27 00:00:00, origina...",[withdrawals],C|2000.0|2KUD65XQ-240326210101 LA LANG
4,490016124,2024-03-18 00:00:00.000000 UTC,2024-03-18 00:00:00.000000 UTC,C,4900.0,NGN,2.819936e+08,NGN,[3@2KUD4RTP2403181934584900.00],withdrawals,...,2024-06-27 07:54:12.000000 UTC,NaN,NaN,[430],"[699813394, 686502308]",NaN,NaN,"{transaction_date=2024-03-18 00:00:00, origina...",[withdrawals],C|4900.0|2KUD4RTP-240318193458 LA LANG


In [ ]:
NombaAccessISW['reporting_tag'].value_counts()

,count
reporting_tag,
withdrawals,972215
chargebacks,25290
fees,1321
prefunding,627
interbank_transfers,524
others,14
investment,9


In [ ]:
import re
def column_extract(df):
  # Define regex patterns for tid and rrn
    tid_pattern = re.compile(r'llm_prediction=([^,}]+)')
    df['llm_prediction'] = df['metadata'].apply(lambda x: tid_pattern.search(x).group(1) if tid_pattern.search(x) else None)
    return df

NombaAcessISW=column_extract(NombaAccessISW)
NombaAcessISW['llm_prediction'].value_counts()
NombaAcessISW[['reporting_tag','is_reversal']].value_counts()


,,count
reporting_tag,is_reversal,
withdrawals,False,963664
chargebacks,False,18258
withdrawals,True,8551
chargebacks,True,7032
fees,False,1321
prefunding,False,627
interbank_transfers,False,521
others,False,11
investment,False,9


In [ ]:
NombaAccessISW

,id,transaction_date,value_date,transaction_type,amount,amount_currency,balance,balance_currency,matching_reference,reporting_tag,...,fsp_accounts,fsp_transaction_ids,backend_accounts,backend_transaction_ids,other_accounts,other_transaction_ids,metadata,tags,description,llm_prediction
0,490914812,2024-03-26 00:00:00.000000 UTC,2024-03-26 00:00:00.000000 UTC,C,5100.0,NGN,6.485760e+08,NGN,[3@2KUD3GWS2403261817305100.00],withdrawals,...,NaN,NaN,[430],[700497534],NaN,NaN,"{transaction_date=2024-03-26 00:00:00, origina...",[withdrawals],C|5100.0|2KUD3GWS-240326181730 LA LANG,None
1,490931395,2024-03-26 00:00:00.000000 UTC,2024-03-26 00:00:00.000000 UTC,C,50000.0,NGN,8.542449e+08,NGN,[3@2KUD121N24032619253050000.00],withdrawals,...,NaN,NaN,[430],[700731674],NaN,NaN,"{transaction_date=2024-03-26 00:00:00, origina...",[withdrawals],C|50000.0|2KUD121N-240326192530 LA LANG,None
2,490943811,2024-03-27 00:00:00.000000 UTC,2024-03-26 00:00:00.000000 UTC,C,5200.0,NGN,2.235371e+08,NGN,[3@2KUD164N2403262040145200.00],withdrawals,...,NaN,NaN,[430],[700356196],NaN,NaN,"{transaction_date=2024-03-27 00:00:00, origina...",[withdrawals],C|5200.0|2KUD164N-240326204014 LA LANG,None
3,490946685,2024-03-27 00:00:00.000000 UTC,2024-03-26 00:00:00.000000 UTC,C,2000.0,NGN,2.617250e+08,NGN,[3@2KUD65XQ2403262101012000.00],withdrawals,...,NaN,NaN,[430],[700335445],NaN,NaN,"{transaction_date=2024-03-27 00:00:00, origina...",[withdrawals],C|2000.0|2KUD65XQ-240326210101 LA LANG,None
4,490016124,2024-03-18 00:00:00.000000 UTC,2024-03-18 00:00:00.000000 UTC,C,4900.0,NGN,2.819936e+08,NGN,[3@2KUD4RTP2403181934584900.00],withdrawals,...,NaN,NaN,[430],"[699813394, 686502308]",NaN,NaN,"{transaction_date=2024-03-18 00:00:00, origina...",[withdrawals],C|4900.0|2KUD4RTP-240318193458 LA LANG,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,494240239,2024-04-30 00:00:00.000000 UTC,2024-04-30 00:00:00.000000 UTC,C,3100.0,NGN,2.719522e+08,NGN,[3@2KUD3ULK2404301205363100.00],withdrawals,...,NaN,NaN,[430],[692546661],NaN,NaN,"{transaction_date=2024-04-30 00:00:00, origina...",[withdrawals],C|3100.0|2KUD3ULK-240430120536 LA LANG,None
999996,494232558,2024-04-30 00:00:00.000000 UTC,2024-04-30 00:00:00.000000 UTC,C,10000.0,NGN,8.235788e+08,NGN,[3@2KUD4IE624043011204810000.00],withdrawals,...,NaN,NaN,[430],[692729379],NaN,NaN,"{transaction_date=2024-04-30 00:00:00, origina...",[withdrawals],C|10000.0|2KUD4IE6-240430112048 LA LANG,None
999997,494196430,2024-04-30 00:00:00.000000 UTC,2024-04-30 00:00:00.000000 UTC,C,15000.0,NGN,4.656227e+08,NGN,[3@2KUD43K324043007091315000.00],withdrawals,...,NaN,NaN,[430],[692558260],NaN,NaN,"{transaction_date=2024-04-30 00:00:00, origina...",[withdrawals],C|15000.0|2KUD43K3-240430070913 LA LANG,None
999998,494250150,2024-04-30 00:00:00.000000 UTC,2024-04-30 00:00:00.000000 UTC,C,54500.0,NGN,4.608035e+08,NGN,[3@2KUD279624043012541654500.00],withdrawals,...,NaN,NaN,[430],[692708626],NaN,NaN,"{transaction_date=2024-04-30 00:00:00, origina...",[withdrawals],C|54500.0|2KUD2796-240430125416 LA LANG,None


In [ ]:
# Cleaning_tag_names
def clean_tag_desc(df):
  df['reporting_tag'] = df['reporting_tag'].str.replace("unknown_pattern","").str.replace("no_ref","").str.replace(r'[^a-zA-Z0-9\s]', '', regex=True).str.lower()
  df['description_con']=df['description'] + "|" + df['is_reversal'].astype('string')
  return df

# Filter data for training
def filter_data(df):
  df=df.loc[(df['reporting_tag']!='others') & (df['description']!= None)]
  df_cleaned=df.dropna(axis=0,subset=['description'])
  return df_cleaned

def filter_data2(df):
  df_cleaned=df.dropna(axis=0,subset=['description'])
  return df_cleaned

NombaAcessISW1=clean_tag_desc(NombaAcessISW)
NombaAcessISW2=filter_data(NombaAcessISW1)
NombaAcessISW3=filter_data2(NombaAcessISW1)


print(NombaAcessISW.shape)
print(NombaAcessISW2.shape)
print(NombaAcessISW3.shape)

(1000000, 31)
(999986, 31)
(1000000, 31)


In [ ]:
display(NombaAcessISW2['reporting_tag'].value_counts().to_frame())
display(NombaAcessISW3['reporting_tag'].value_counts().to_frame())

,count
reporting_tag,
withdrawals,972215
chargebacks,25290
fees,1321
prefunding,627
interbanktransfers,524
investment,9


,count
reporting_tag,
withdrawals,972215
chargebacks,25290
fees,1321
prefunding,627
interbanktransfers,524
others,14
investment,9


In [ ]:
NombaAcessISW3['description']=NombaAcessISW3['description'].str.lower()

BERT uses:
1. Fine tuning approach
2. Feature based approach
3. As word embedding

For fine tuning, take the last layer Next Sentence Prediction-Dense and tie its output to a new fully connected dense layer,as shown below.
Recommendations
 - Batch size: 16, 32
 - Learning rate (Adam): 5e-5, 3e-5, 2e-5
 - Number of epochs: 2, 3, 4

In [ ]:
!pip install transformers

In [ ]:
#hide
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Imports
!pip install datasets
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.nn.utils import clip_grad_norm_
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm


# # compute metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming NombaAcessISW2 is the DataFrame you want to split
train_df, temp_df = train_test_split(NombaAcessISW2, test_size=0.3, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.67, random_state=42)

# Reset the indices of the DataFrames
train_df.reset_index(drop=True, inplace=True)
valid_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

print(f"Length of train_df: {len(train_df)}")
print(f"Length of valid_df: {len(valid_df)}")
print(f"Length of test_df: {len(test_df)}")

Length of train_df: 699990
Length of valid_df: 98998
Length of test_df: 200998


TOKENIZATION

In [ ]:
# Initialize a bert tokenizer

from transformers import BertTokenizer
PRETRAINED_LM = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_LM, do_lower_case=True)
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
# Encoding function
def encode(docs):
    '''
    This function takes list of texts and returns input_ids and attention_mask of texts
    '''
    encoded_dict = tokenizer.batch_encode_plus(docs, add_special_tokens=True, max_length=128, padding='max_length',
                            return_attention_mask=True, truncation=True, return_tensors='pt')
    input_ids = encoded_dict['input_ids']
    attention_masks = encoded_dict['attention_mask']
    return input_ids, attention_masks

In [ ]:
# use encode function to get input ids and attention masks of the datasets

# may take a while to execute if on ram
train_input_ids, train_att_masks = encode(train_df['description'].values.tolist())
valid_input_ids, valid_att_masks = encode(valid_df['description'].values.tolist())
test_input_ids, test_att_masks = encode(test_df['description'].values.tolist())

In [ ]:
NombaAcessISW3.columns

Index(['id', 'transaction_date', 'value_date', 'transaction_type', 'amount',
       'amount_currency', 'balance', 'balance_currency', 'matching_reference',
       'reporting_tag', 'is_reversal', 'is_duplicate',
       'is_duplicate_parent_reference', 'matched', 'reversal', 'client_id',
       'account_id', 'batch_id', 'created', 'updated', 'fsp_accounts',
       'fsp_transaction_ids', 'backend_accounts', 'backend_transaction_ids',
       'other_accounts', 'other_transaction_ids', 'metadata', 'tags',
       'description', 'llm_prediction', 'description_con'],
      dtype='object')

In [ ]:
NombaAcessISW3['reporting_tag'].value_counts()

,count
reporting_tag,
withdrawals,972215
chargebacks,25290
fees,1321
prefunding,627
interbanktransfers,524
others,14
investment,9


In [ ]:
# Create a dictionary to map reporting_tag values to numbers
tag_mapping = {
    'withdrawals': 1,
    'chargebacks': 2,
    'fees': 3,
    'prefunding': 4,
    'interbanktransfers': 5,
    'others': 6,
    'investment': 7
}

# Apply the mapping to create the new column 'reporting_tag_numeric'
train_df['reporting_tag_numeric'] = train_df['reporting_tag'].map(tag_mapping)
valid_df['reporting_tag_numeric'] = valid_df['reporting_tag'].map(tag_mapping)
test_df['reporting_tag_numeric'] = test_df['reporting_tag'].map(tag_mapping)

In [ ]:
import torch
train_y = torch.LongTensor(train_df['reporting_tag_numeric'].values.tolist())
valid_y = torch.LongTensor(valid_df['reporting_tag_numeric'].values.tolist())
test_y = torch.LongTensor(test_df['reporting_tag_numeric'].values.tolist())
train_y.size(),valid_y.size(),test_y.size()

(torch.Size([699990]), torch.Size([98998]), torch.Size([200998]))

In [ ]:
# Create dataloaders for training
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


# larger batch size for faster execution
BATCH_SIZE = 32

# Create dataloaders for training
train_dataset = TensorDataset(train_input_ids, train_att_masks, train_y)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

valid_dataset = TensorDataset(valid_input_ids, valid_att_masks, valid_y)
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_input_ids, test_att_masks, test_y)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)


Initiate BERT sequence classification model allowing easy fine-tuning the pretrained BERT mode for classification task.

In [ ]:
from transformers import BertForSequenceClassification
N_reporting_tag_numerc = len(train_df.reporting_tag_numeric.unique())
model = BertForSequenceClassification.from_pretrained(PRETRAINED_LM,
                                                      num_labels=N_reporting_tag_numerc,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model = model.cuda()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


FINE TUNING

In [ ]:
# Optimizer to tune parameters that set up a learning rate
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

EPOCHS = 11
LEARNING_RATE = 1e-4

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer,
             num_warmup_steps=0,
            num_training_steps=len(train_dataloader)*EPOCHS )

TRAINING LOOP

In [ ]:
import numpy as np
#collapse-output
from torch.nn.utils import clip_grad_norm_
from tqdm.notebook import tqdm
#import numpy as np
import math

train_loss_per_epoch = []
val_loss_per_epoch = []


# Training section
for epoch_num in range(EPOCHS):
    print('Epoch: ', epoch_num + 1)
    '''
    Training
    '''
    model.train()
    train_loss = 0
    for step_num, batch_data in enumerate(tqdm(train_dataloader,desc='Training')):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

        loss = output.loss
        train_loss += loss.item()

        model.zero_grad()
        loss.backward()
        del loss

        clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    train_loss_per_epoch.append(train_loss / (step_num + 1))

# Validation section
    '''
    Validation
    '''
    model.eval()
    valid_loss = 0
    valid_pred = []
    with torch.no_grad():
        for step_num_e, batch_data in enumerate(tqdm(valid_dataloader,desc='Validation')):
            input_ids, att_mask, labels = [data.to(device) for data in batch_data]
            output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

            loss = output.loss
            valid_loss += loss.item()

            valid_pred.append(np.argmax(output.logits.cpu().detach().numpy(),axis=-1))

    val_loss_per_epoch.append(valid_loss / (step_num_e + 1))
    valid_pred = np.concatenate(valid_pred)

    '''
    Loss message
    '''
    print("{0}/{1} train loss: {2} ".format(step_num+1, math.ceil(len(train_df) / BATCH_SIZE), train_loss / (step_num + 1)))
    print("{0}/{1} val loss: {2} ".format(step_num_e+1, math.ceil(len(valid_df) / BATCH_SIZE), valid_loss / (step_num_e + 1)))

In [ ]:
# Visualize training and validation losses for each epnoch
from matplotlib import pyplot as plt
epochs = range(1, EPOCHS +1 )
fig, ax = plt.subplots()
ax.plot(epochs,train_loss_per_epoch,label ='training loss')
ax.plot(epochs, val_loss_per_epoch, label = 'validation loss' )
ax.set_title('Training and Validation loss')
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
ax.legend()
plt.show()

Performance metrics

Looking at :    
  - precision
  - recall
  - f1 score

In [ ]:
from sklearn.metrics import classification_report
print('classifiation report')
print(classification_report(valid_pred, valid_df['description_tag_numeric'].to_numpy(), target_names=description_tag_names))

In [ ]:
# Error analysis using confusion matrix
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
def plot_confusion_matrix(y_preds, y_true, labels=None):
  cm = confusion_matrix(y_true, y_preds, normalize="true")
  fig, ax = plt.subplots(figsize=(6, 6))
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
  disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
  plt.title("Normalized confusion matrix")
  plt.show()

PREDICTION
Use the trained model to predict the testing set (20%)

In [ ]:
model.eval()
test_pred = []
test_loss= 0
with torch.no_grad():
    for step_num, batch_data in tqdm(enumerate(test_dataloader)):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

        loss = output.loss
        test_loss += loss.item()

        test_pred.append(np.argmax(output.logits.cpu().detach().numpy(),axis=-1))
test_pred = np.concatenate(test_pred)

In [ ]:
print('classifiation report')
print(classification_report(test_pred, test_df['description_tag_numeric'].to_numpy(),target_names=label_names))

In [ ]:
plot_confusion_matrix(test_pred,test_df['description_tag_numeric'].to_numpy(),labels=label_names)

In [ ]:
test_df['pred'] = test_pred
test_df.reset_index(level=0)
print(test_df[test_df['label']!=test_df['pred']].shape)
test_df[test_df['label']!=test_df['pred']][['text','label','pred']].head(10)

In [ ]:
#hide
test_df.to_csv("test_results.csv",index= False)